In [11]:
import os
import numpy

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model

from sklearn.externals.joblib import dump
from sklearn.externals.joblib import load
from sklearn.preprocessing import MinMaxScaler

class NeuralNetwork(object):
    def __init__(self, data_file=None, epochs=500, batch=32, activation='sigmoid', loss='binary_crossentropy', optimizer='adam', split=0.20):
        self.data_file = data_file
        self.features = None
        self.labels = None
        self.epochs = epochs
        self.batch = batch
        self.scaler = MinMaxScaler(feature_range=(0, 1))

        self.clf = Sequential(name='Sandbox')       
        self.clf.add(Dense(3, input_dim=3, name='Input'))
        self.clf.add(Dense(3, activation=activation, name='Hidden_layer_1'))
        self.clf.add(Dense(3, activation=activation, name='Hidden_layer_2'))
        self.clf.add(Dense(1, activation=activation, name='Output'))
        self.clf.compile(loss=loss, optimizer=optimizer, metrics=['acc'])
        self.split = split

    def load_data(self):
        dataset = numpy.loadtxt(self.data_file, delimiter=",", dtype='float')
        self.features = dataset[:,0:3]
        self.labels = dataset[:,3]

    def scale_data(self, save_scaler=True):
        self.features = self.features.astype(numpy.float)
        self.labels = self.labels.astype(numpy.float)
        self.labels.reshape(-1, 1)

        self.scaler.fit(self.features)
        self.min_max_data = self.scaler.transform(self.features)

        if save_scaler:
            dump(self.scaler, 'minmax_scaler.bin', compress=True)

    def train(self, metrics=True):
        self.clf.fit(self.min_max_data, self.labels, epochs=self.epochs, batch_size=self.batch, validation_split=self.split, verbose=False)

        if metrics:
            scores = self.clf.evaluate(self.min_max_data, self.labels)
            print("\n%s: %.2f%%" % (self.clf.metrics_names[1], scores[1]*100))

    def predict(self, features, verbose=True):
        # test = numpy.array([[131, 65.5, 2]])
        # test1 = numpy.array([[36, 18.0, 2]])

        features = features.astype(numpy.float64) 
        min_max_data = self.scaler.transform(features)

        if verbose:
            print(f'Predicted Class: {self.clf.predict_classes(min_max_data)[0][0]}')
            print(f'Class Probability: {self.clf.predict(min_max_data)[0][0]}')

    def save(self, outfile):
        self.model.save(outfile, save_format='tf')

    def load(self, model_path, scaler_file):
        self.model = load_model(model_path)
        self.scaler_file = load(scaler_file)




In [12]:
nn = NeuralNetwork(data_file=".\\data\\data_process_count.csv")

In [13]:
nn.load_data()

In [14]:
nn.scale_data()

In [15]:
nn.train()

384/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [16]:
safe_test = numpy.array([[131, 65.5, 2]])
nn.predict(safe_test)

Predicted Class: 0
Class Probability: 0.00819447636604309


In [17]:
unsafe_test = numpy.array([[36, 18.0, 2]])
nn.predict(unsafe_test)

Predicted Class: 1
Class Probability: 0.6846142411231995
